# Descriptive analysis (per image)

This notebook builds per-image metrics by combining:
- feature_engineering_summary.csv (file-level features)
- Raw fixation CSVs in the fixations folder (for pupil metrics)

Outputs a per-image table with:
- number_of_fixations (sum over files/participants)
- fixation_duration_mean_weighted, fixation_duration_median_approx
- view_time_total_sum, scanpath_length_mean, BCEA_68_mean, BCEA_95_mean
- pupil metrics per image (mean and std for each available pupil column)


In [1]:
# Setup and paths
import os
from pathlib import Path
import pandas as pd
import numpy as np

# Resolve project root (assumes this file in data_analysis/descriptive_analysis)
nb_dir = Path.cwd()
project_root = nb_dir.parents[2] if len(nb_dir.parents) >= 2 else nb_dir

summary_candidates = [
    nb_dir.parent / "feature_engineering" / "feature_engineering_summary.csv",
    project_root / "data_analysis" / "feature_engineering" / "feature_engineering_summary.csv",
]
summary_path = next((p for p in summary_candidates if p.exists()), None)
if summary_path is None:
    raise FileNotFoundError("feature_engineering_summary.csv not found.")

fixations_candidates = [
    project_root / "fixations",
    nb_dir.parents[3] / "fixations" if len(nb_dir.parents) >= 3 else nb_dir / "fixations",
    Path(r"c:\\Users\\SWixforth\\Uni\\eye-tracking-ai\\fixations"),
]
fixations_dir = next((p for p in fixations_candidates if p.exists()), None)
if fixations_dir is None:
    raise FileNotFoundError("fixations folder not found.")

print(f"Using summary: {summary_path}\nUsing fixations: {fixations_dir}")


Using summary: c:\Users\SWixforth\Uni\eye-tracking-ai\data_analysis\feature_engineering\feature_engineering_summary.csv
Using fixations: c:\Users\SWixforth\Uni\eye-tracking-ai\fixations


In [2]:
# Load feature_engineering summary
summary = pd.read_csv(summary_path)

# Ensure image_id is string with zero padding as in filenames
summary["image_id"] = summary["image_id"].astype(str).str.zfill(3)

# Per-image aggregates from summary
per_image_summary = (
    summary.groupby("image_id").agg(
        number_of_fixations=("n_fix", "sum"),
        view_time_total_sum=("view_time_total", "sum"),
        fixation_duration_mean_weighted=("fix_dur_mean", "mean"),
        fixation_duration_median_approx=("fix_dur_median", "median"),
        scanpath_length_mean=("scanpath_length", "mean"),
        BCEA_68_mean=("bcea_68", "mean"),
        BCEA_95_mean=("bcea_95", "mean"),
        primary_label_top=("primary_label", lambda s: s.mode().iloc[0] if not s.mode().empty else np.nan),
    )
    .reset_index()
)

per_image_summary.head()

,image_id,number_of_fixations,view_time_total_sum,fixation_duration_mean_weighted,fixation_duration_median_approx,scanpath_length_mean,BCEA_68_mean,BCEA_95_mean,primary_label_top
0,001,1159,425375.042,284.329020,232.36300,3688.463661,72008.690066,189496.552805,meme
1,002,1284,452560.267,274.577679,232.92700,2665.561573,60359.429235,158840.603249,meme
2,003,1110,411435.771,296.863510,245.25175,3022.834081,40009.274302,105287.563953,meme
3,004,1354,469790.081,280.162546,216.58800,3536.890776,87693.362497,230772.006571,meme
4,005,1359,460515.114,265.496762,216.43100,3734.527053,90696.081085,238673.897592,meme


In [3]:
# Build pupil metrics per image from raw fixations
import glob
import re

fname_re = re.compile(r"^P(?P<participant>\d+)_id(?P<image>\d+).+\.csv$")

pupil_rows = []
for fp in glob.glob(str(fixations_dir / "*.csv")):
    name = os.path.basename(fp)
    m = fname_re.match(name)
    if not m:
        continue
    image_id = m.group("image")
    image_id = str(image_id).zfill(3)
    try:
        df = pd.read_csv(fp)
    except Exception as e:
        print(f"Failed to read {name}: {e}")
        continue

    # Identify pupil columns (common names: pupil, pupil_left/right, pupil_size, pupil_diameter)
    pupil_cols = [c for c in df.columns if c.lower().startswith("pupil")] 
    if not pupil_cols:
        continue

    stats = {"image_id": image_id}
    for c in pupil_cols:
        s = pd.to_numeric(df[c], errors="coerce")
        stats[f"{c}_mean"] = float(s.mean())
        stats[f"{c}_std"] = float(s.std(ddof=1)) if s.count() > 1 else np.nan
    pupil_rows.append(stats)

pupil_df = pd.DataFrame(pupil_rows)
# Reduce to per-image by averaging across files if multiple files per image
if not pupil_df.empty:
    agg_map = {col: "mean" for col in pupil_df.columns if col != "image_id"}
    pupil_per_image = pupil_df.groupby("image_id").agg(agg_map).reset_index()
else:
    pupil_per_image = pd.DataFrame(columns=["image_id"])  # empty

pupil_per_image.head()

,image_id,pupil_size_norm_mean,pupil_size_norm_std
0,001,2.101475e-16,1.0
1,002,-2.377767e-16,1.0
2,003,2.772367e-16,1.0
3,004,-7.328157e-17,1.0
4,005,-4.174451e-17,1.0


### Pupil data computation (overview)
- Inputs (from Tobii export): `left_pupil_diameter`, `right_pupil_diameter`.
- Cleaning: values outside the plausible human range [1.5, 8] mm → NaN; non‑numeric/corrupted entries → NaN.
- Combine eyes per sample:
  - both valid → average(left, right)
  - one valid → use that eye
  - none valid → NaN
- Sanity pass: mask the combined average again if outside [1.5, 8] mm.
- Interpolation: short gaps in `pupil_size` were linearly interpolated (together with x and y) to bridge blinks/dropped samples.
- Output: cleaned, interpolated `pupil_size` alongside x, y, and a millisecond timestamp (relative to the first frame).

Normalization and negatives
- `pupil_size_norm` is a normalized series (e.g., z‑score or baseline‑relative change).
- Per‑fixation means like `pupil_size_norm_mean` can be negative (below baseline/mean) or positive (above).
- For the exact normalization used here, see the code around cell 4 in this notebook.

In [4]:
# Time dynamics: first vs last third mean fixation duration per image
import numpy as np

time_rows = []
for fp in glob.glob(str(fixations_dir / "*.csv")):
    name = os.path.basename(fp)
    m = fname_re.match(name)
    if not m:
        continue
    image_id = str(m.group("image")).zfill(3)
    try:
        df = pd.read_csv(fp)
    except Exception as e:
        print(f"Failed to read {name}: {e}")
        continue
    # Need start_time, end_time, duration; derive end_time if missing
    if not {"start_time","end_time","duration"}.issubset(df.columns):
        if "start_time" in df.columns and "duration" in df.columns:
            df = df.copy()
            df["end_time"] = pd.to_numeric(df["start_time"], errors="coerce") + pd.to_numeric(df["duration"], errors="coerce")
        else:
            continue
    st = pd.to_numeric(df["start_time"], errors="coerce")
    et = pd.to_numeric(df["end_time"], errors="coerce")
    dur = pd.to_numeric(df["duration"], errors="coerce")
    mask_valid = st.notna() & et.notna() & dur.notna()
    if not mask_valid.any():
        continue
    st, et, dur = st[mask_valid], et[mask_valid], dur[mask_valid]
    t0, t1 = st.min(), et.max()
    if not np.isfinite(t0) or not np.isfinite(t1) or t1 <= t0:
        continue
    b1 = t0 + (t1 - t0) / 3.0
    b2 = t0 + 2.0 * (t1 - t0) / 3.0
    mid = (st + et) / 2.0
    first_mask = mid < b1
    last_mask = mid >= b2
    first_mean = float(dur[first_mask].mean()) if first_mask.any() else np.nan
    last_mean = float(dur[last_mask].mean()) if last_mask.any() else np.nan
    n_first = int(first_mask.sum())
    n_last = int(last_mask.sum())
    time_rows.append({
        "image_id": image_id,
        "fix_dur_mean_first_third": first_mean,
        "fix_dur_mean_last_third": last_mean,
        "n_fix_first_third": n_first,
        "n_fix_last_third": n_last,
    })

time_dyn_df = pd.DataFrame(time_rows)
if not time_dyn_df.empty:
    time_dyn_per_image = (
        time_dyn_df.groupby("image_id").agg({
            "fix_dur_mean_first_third": "mean",
            "fix_dur_mean_last_third": "mean",
            "n_fix_first_third": "sum",
            "n_fix_last_third": "sum",
        }).reset_index()
    )
else:
    time_dyn_per_image = pd.DataFrame(columns=[
        "image_id","fix_dur_mean_first_third","fix_dur_mean_last_third","n_fix_first_third","n_fix_last_third"
    ])

time_dyn_per_image.head()

,image_id,fix_dur_mean_first_third,fix_dur_mean_last_third,n_fix_first_third,n_fix_last_third
0,001,274.590950,311.450110,406,378
1,002,270.570046,286.048076,440,434
2,003,261.323237,375.535697,415,350
3,004,269.514004,306.053531,477,445
4,005,255.631797,281.616679,480,435


In [5]:
# Join per-image summary with pupil metrics and time dynamics
per_image = per_image_summary.merge(pupil_per_image, on="image_id", how="left")
per_image = per_image.merge(time_dyn_per_image, on="image_id", how="left")

out_csv = nb_dir / "per_image_descriptive_summary.csv"
per_image.to_csv(out_csv, index=False)
print(f"Saved per-image descriptive summary: {out_csv}")
per_image.head(10)

Saved per-image descriptive summary: c:\Users\SWixforth\Uni\eye-tracking-ai\data_analysis\descriptive_analysis\per_image_descriptive_summary.csv


,image_id,number_of_fixations,view_time_total_sum,fixation_duration_mean_weighted,fixation_duration_median_approx,scanpath_length_mean,BCEA_68_mean,BCEA_95_mean,primary_label_top,pupil_size_norm_mean,pupil_size_norm_std,fix_dur_mean_first_third,fix_dur_mean_last_third,n_fix_first_third,n_fix_last_third
0,001,1159,425375.042,284.329020,232.36300,3688.463661,72008.690066,189496.552805,meme,2.101475e-16,1.0,274.590950,311.450110,406,378
1,002,1284,452560.267,274.577679,232.92700,2665.561573,60359.429235,158840.603249,meme,-2.377767e-16,1.0,270.570046,286.048076,440,434
2,003,1110,411435.771,296.863510,245.25175,3022.834081,40009.274302,105287.563953,meme,2.772367e-16,1.0,261.323237,375.535697,415,350
3,004,1354,469790.081,280.162546,216.58800,3536.890776,87693.362497,230772.006571,meme,-7.328157e-17,1.0,269.514004,306.053531,477,445
4,005,1359,460515.114,265.496762,216.43100,3734.527053,90696.081085,238673.897592,meme,-4.174451e-17,1.0,255.631797,281.616679,480,435
5,006,1209,414304.850,260.881673,232.86000,3376.023058,106973.130908,281508.239230,meme,1.145180e-16,1.0,250.958306,287.164563,440,375
6,007,1224,429382.963,278.408482,216.72750,3953.095336,79180.770017,208370.447414,meme,5.300824e-16,1.0,262.911944,292.221539,436,394
7,008,1108,433119.649,318.751568,232.58825,2988.267206,97520.447439,256632.756419,meme,7.728491e-16,1.0,313.981583,358.660547,378,362
8,009,1417,461028.574,262.610138,232.71500,3678.842286,111612.600233,293717.369034,meme,-5.038867e-17,1.0,244.313509,273.930288,512,449
9,010,1469,510861.931,288.140492,241.23400,2811.817560,55985.496083,147330.252850,meme,-1.863976e-16,1.0,275.804482,289.634173,530,474


In [6]:
# Quick sanity check tables
print("Per-image counts and durations (head):")
print(per_image[[
    "image_id","number_of_fixations","view_time_total_sum","fixation_duration_mean_weighted","fixation_duration_median_approx"
]].head())

num_cols = [c for c in per_image.columns if per_image[c].dtype != "O" and c != "image_id"]
per_image[num_cols].describe().T

Per-image counts and durations (head):
  image_id  number_of_fixations  view_time_total_sum  \
0      001                 1159           425375.042   
1      002                 1284           452560.267   
2      003                 1110           411435.771   
3      004                 1354           469790.081   
4      005                 1359           460515.114   

   fixation_duration_mean_weighted  fixation_duration_median_approx  
0                       284.329020                        232.36300  
1                       274.577679                        232.92700  
2                       296.863510                        245.25175  
3                       280.162546                        216.58800  
4                       265.496762                        216.43100  


,count,mean,std,min,25%,50%,75%,max
number_of_fixations,152.0,1.413250e+03,3.420339e+02,9.440000e+02,1.200500e+03,1.320500e+03,1.570250e+03,2.556000e+03
view_time_total_sum,152.0,4.644925e+05,6.684295e+04,3.604084e+05,4.213195e+05,4.470682e+05,4.812201e+05,6.915470e+05
fixation_duration_mean_weighted,152.0,2.677860e+02,3.558548e+01,1.960005e+02,2.388399e+02,2.745366e+02,2.953214e+02,3.510048e+02
fixation_duration_median_approx,152.0,2.233594e+02,2.155692e+01,1.824420e+02,1.999045e+02,2.325068e+02,2.410259e+02,2.662170e+02
scanpath_length_mean,152.0,3.192534e+03,7.188735e+02,1.933301e+03,2.699730e+03,3.018405e+03,3.587123e+03,5.197240e+03
BCEA_68_mean,152.0,8.638422e+04,2.943021e+04,1.913083e+04,6.520967e+04,8.352358e+04,1.039594e+05,1.870754e+05
BCEA_95_mean,152.0,2.273269e+05,7.744792e+04,5.034428e+04,1.716044e+05,2.197989e+05,2.735773e+05,4.923037e+05
pupil_size_norm_mean,152.0,-3.281194e-17,3.928290e-16,-1.111996e-15,-2.525365e-16,-3.605276e-18,2.119864e-16,1.134850e-15
pupil_size_norm_std,152.0,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
fix_dur_mean_first_third,152.0,2.539455e+02,3.797449e+01,1.908019e+02,2.165020e+02,2.599550e+02,2.829442e+02,3.328939e+02


## Notes
- All metrics are aggregated per image_id across all participants/files.
- Pupil metrics were computed from raw fixations CSVs and averaged per image.
- Time dynamics (first vs last third) is computed using fixation midpoints within each image’s total viewing span.
- If no pupil columns exist in the fixations files, those columns will be missing (NaN).
